<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/ChatGPT%20app/5_1_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("{topic}에 대한 간단한 농담을 말해줘.")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser # LCEL의 기본 파이프라인

chain.invoke({"topic": "아이스크림"})

'아이스크림이 왜 항상 친구가 되고 싶어 할까? \n\n- 아이스크림: "나랑 친구가 되면, 항상 달콤한 시간을 보내게 될 거야!"'

In [ ]:
!pip install langchain docarray tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["해리슨은 학교에서 일합니다.", "곰은 꿀을 좋아해"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """다음 지문에만 근거해서 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel( # 여러개의 Runnable을 병렬적으로 실행
    {"context": retriever, "question": RunnablePassthrough()} # RunnablePassthrough는 값을 입력받아 그대로 전달하는 객체
)
chain = setup_and_retrieval | prompt | model | output_parser

# retrieval | prompt | model | output. RAG의 기본 파이프라인

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
chain.invoke("해리슨은 어디에서 일하나요?")

'답변: 학교에서 일합니다.'

In [ ]:

chain.invoke("곰은 무엇을 좋아하나요?")

'답변: 곰은 꿀을 좋아합니다.'

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 간단한 농담을 해줘."
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

왜 아이스크림은 항상 차가운가요? 

- 아이스크림이 뜨거우면 녹잖아요!

In [ ]:
chain.batch(["아이스크림", "스파게티", "만두"])

# stream: 입력에 대해 청크를 스트리밍
# invoke: 입력에 대해 체인을 호출
# batch: 입력목록에 대해 체인을 배치로 호출

['어떤 아이스크림이 가장 시끄럽나요? \n\n소리크림!',
 '스파게티가 자꾸 말을 걸어오면 어쩌죠? \n\n너무 끌려서 안 먹게 될 것 같아요!',
 '만두가 뭐라고 미안해하니? "나만두지 말고 다 같이 먹자!"']

In [ ]:
await chain.ainvoke("ice cream")

# astream: 비동기로 입력에 대해 청크를 스트리밍
# ainvoke: 비동기로 입력에 대해 체인을 호출
# abatch: 비동기로 입력목록에 대해 체인을 배치로 호출

'어떤 아이스크림이 가장 추억에 남아? 비주얼 아이스크림!'

In [ ]:
await chain.abatch(
    ["아이스크림", "스파게티", "만두"]
)

['어떤 아이스크림이 가장 신나게 춤을 추나요? 바닐라 아이스크림! 왜냐하면 그녀는 밀크쉐이크를 좋아해서요!',
 "스파게티는 왜 항상 즐겁게 먹을까? \n\n왜냐하면 항상 너무 '파게티'하기 때문이야! 😆🍝",
 '왜 만두가 자꾸 먹히냐고? 만두가 자꾸 만둣군요!']

In [ ]:
chain.input_schema.schema()

{'title': 'RunnableParallel<topic>Input', 'type': 'object', 'properties': {}}

In [ ]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}